<a href="https://colab.research.google.com/github/nupursjsu/Deep-Learning/blob/master/Graded_Assignment4/VGG16_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementing VGG16 in PyTorch (Using CIFAR 100 dataset)

## Importing required libraries

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F

## Loading CIFAR 100 dataset

In [0]:
def load_data(train_batch_size, test_batch_size):
    #Fetching test data containing 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

    #Fetching test data containing 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('data', train=False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader, test_loader)

##Building VGG16 model

In [0]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        #Defining fully connected layers
        self.fc6 = nn.Linear(2048, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x

##Defining the train function

In [0]:
def train(model, optimizer, epoch, train_loader, log_interval):
    model.train()

    #Defining loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    #Iterating over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data,target = data.to(device='cuda'),target.to(device='cuda')
        data, target = Variable(data), Variable(target)

        #Setting the gradients to zero, since PyTorch accumulates them
        optimizer.zero_grad()

        #Defining Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        #Defining Backward propagation
        loss.backward()

        #Updating the parameters after backward prop
        optimizer.step()

        #Displaying the log
        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.item()))

##Defining the test function

In [0]:
def test(model, epoch, test_loader):
    #Defining that we are testing the model
    model.eval()

    #Initializing the variables for init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    #Defining loss function
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    #Iterating over data
    for data, target in test_loader:
        data,target = data.to(device='cuda'),target.to(device='cuda')
        data, target = Variable(data), Variable(target)
        
        #Defining Forward propagation
        output = model(data)

        #Calculating & accumulating loss
        test_loss += loss_fn(output, target).item()

        #Getting the index of the max log-probability which is the predicted output label
        pred = np.argmax(output.cpu().data, axis=1)

        #If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.cpu().data).sum()

    #Displaying the log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

##Defining the parameters

In [0]:
torch.manual_seed(123)

model = VGG16()
model.cuda()
lr = 0.01
momentum=0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_batch_size = 64
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

Extracting data/cifar-100-python.tar.gz to data


##Training and Testing the VGG16 model

In [0]:
epochs = 10
log_interval = 100
for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/50000 (0%)]	Loss: 4.746198
Train set, Epoch 1 [6400/50000 (13%)]	Loss: 4.643956
Train set, Epoch 1 [12800/50000 (26%)]	Loss: 4.617430
Train set, Epoch 1 [19200/50000 (38%)]	Loss: 4.672313
Train set, Epoch 1 [25600/50000 (51%)]	Loss: 4.662653
Train set, Epoch 1 [32000/50000 (64%)]	Loss: 4.636252
Train set, Epoch 1 [38400/50000 (77%)]	Loss: 4.615027
Train set, Epoch 1 [44800/50000 (90%)]	Loss: 4.661815


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 4.6359, Accuracy: 103/10000 (1%)

Train set, Epoch 2 [0/50000 (0%)]	Loss: 4.618440
Train set, Epoch 2 [6400/50000 (13%)]	Loss: 4.620255
Train set, Epoch 2 [12800/50000 (26%)]	Loss: 4.613917
Train set, Epoch 2 [19200/50000 (38%)]	Loss: 4.620035
Train set, Epoch 2 [25600/50000 (51%)]	Loss: 4.650848
Train set, Epoch 2 [32000/50000 (64%)]	Loss: 4.612297
Train set, Epoch 2 [38400/50000 (77%)]	Loss: 4.649519
Train set, Epoch 2 [44800/50000 (90%)]	Loss: 4.596861

Test set, Epoch 2 , Average loss: 4.6382, Accuracy: 97/10000 (1%)

Train set, Epoch 3 [0/50000 (0%)]	Loss: 4.659497
Train set, Epoch 3 [6400/50000 (13%)]	Loss: 4.615355
Train set, Epoch 3 [12800/50000 (26%)]	Loss: 4.628443
Train set, Epoch 3 [19200/50000 (38%)]	Loss: 4.608543
Train set, Epoch 3 [25600/50000 (51%)]	Loss: 4.607243
Train set, Epoch 3 [32000/50000 (64%)]	Loss: 4.646716
Train set, Epoch 3 [38400/50000 (77%)]	Loss: 4.646516
Train set, Epoch 3 [44800/50000 (90%)]	Loss: 4.643054

Test set, 

##Conclusion

We could see VGG16 on CIFAR 10 dataset is giving a very low accuracy of just 1%.